# Creating a fiber and running a simulation
This tutorial will go over the basics of using wmglab-neuron to create a NEURON model of a peripheral nerve axon (fiber), creating a stimulation waveform, and running a simulation of the fiber's response to the stimulation.

## Creating a fiber
For this tutorial, we will create a fiber model using the MRG model with interpolation. We will also specify the diameter of the fiber to be 5.7 microns.
The `FiberModel` class is used to create a fiber model based on the parameters you specify.
We can determine the length of a fiber by either specifying the number of sections (in which case the length will be calculated), or the length of the fiber in microns (in which case the number of sections will be calculated).


see documentation on fiber models for more information on the different models and their parameters

In [ ]:
from wmglab_neuron import FiberBuilder, FiberModel

n_sections = 133
fiber = FiberBuilder.generate(FiberModel.MRG_INTERPOLATION, diameter=5.7, n_sections=n_sections)

This produces a fiber object that can be used to run simulations.

In [ ]:
print(fiber)
# TODO: add a block going over use of magic methods

## Simulation setup

Before we can run a simulation, we need to create a stimulation waveform. We will use a biphasic square wave for this tutorial.

See documentation on stimulation waveforms for more information on creating different waveforms.

In [ ]:
import numpy as np

waveform = np.concatenate((np.ones(200), -np.ones(200), np.zeros(49600)))

Let's plot the waveform to see what it looks like.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(waveform[:1000])
plt.title('Stimulation waveform')
plt.xlabel('Time step')
plt.ylabel('Normalized Amplitude')
plt.show()

We also need extracellular potentials along the fiber. We will use a normal distribution for this tutorial. Our fiber has 133 sections, so we will create 133 potentials.

See documentation on extracellular potentials for more information on extracellular potentials.

In [ ]:
from scipy.stats import norm

potentials = norm.pdf(np.linspace(-1, 1, n_sections), 0, 0.05) * 100

plt.plot(potentials)
plt.title('Extracellular potentials')
plt.xlabel('Fiber section')
plt.ylabel('Potential (mV)')
plt.show()
# todo change to be from a point source

## Running a simulation

Let's specify our simulation parameters.

In [ ]:
time_step = 0.001  # milliseconds
time_stop = 50  # milliseconds

To run a simulation, we need to create an instance of the Stimulation class. This class is used to run simulations of a fiber's response to a stimulation waveform.

In [ ]:
from wmglab_neuron import Stimulation

# Create instance of Stimulation class
stimulation = Stimulation(fiber, waveform=waveform, potentials=potentials, dt=time_step, tstop=time_stop)
print(stimulation)

We can use the `run_sim()` method to run a simulation. By default, this monitors for action potentials at the distal end of the fiber. `run_sim()` returns the number of detected action potentials and the time of the last action potential.

In [ ]:
# TODO: details about scaling by unit stimulus (1mA) and polarity
ap, time = stimulation.run_sim(-1)
print(f'Number of action potentials detected: {ap}')
print(f'Time of last action potential detection: {time}')

See also the tutorial for [analyzing results](3_analysis.ipynb).